# Pauli Propagation of Parametrized Circuits: Tutorial

This tutorial walks through the core workflow of `pprop`, a library for computing quantum circuit expectation values analytically via **Pauli propagation**.

## What is Pauli Propagation?

Given a parametrized quantum circuit $U(\boldsymbol{\theta})$ and an observable $O$, the expectation value is:

$$\langle O \rangle(\boldsymbol{\theta}) = \langle 0 | U^\dagger(\boldsymbol{\theta})\, O\, U(\boldsymbol{\theta}) | 0 \rangle$$

Rather than simulating the state $U(\boldsymbol{\theta})|0\rangle$ (Schrödinger picture), Pauli propagation works in the **Heisenberg picture**: it propagates the observable $O$ *backwards* through the circuit, gate by gate, expressing the result as a sum of Pauli words with trigonometric coefficients in $\boldsymbol{\theta}$.

The final expectation value is a closed-form trigonometric polynomial, no statevector simulation needed.

## Workflow overview

1. Define the ansatz as a standard PennyLane function
2. Wrap it in a `Propagator` and call `.propagate()`
3. Evaluate expectations and gradients at any parameter point

## Step 0: Imports

We only need PennyLane to define the circuit and `pprop` to propagate it.

In [1]:
import pennylane as qml # To define the circuit
from pprop.propagator import Propagator # For Pauli Propagation

/home/samonaco/Pauli-Propagator/.venv/lib/python3.12/site-packages/pennylane/operation.py:2622: PennyLaneDeprecationWarning: Observable is deprecated and will be removed in v0.43. A generic Operator class should be used instead. If defining an Operator, set the is_hermitian property to True. If checking if an Operator is Hermitian, check the is_hermitian property. 
  warnings.warn(


## Step 1: Define the ansatz

The ansatz must be a plain Python function that:
- accepts a single array-like `params` (parameter values are accessed by integer index)
- applies PennyLane gates using those parameters
- **returns a list of `qml.expval(...)` calls** for every observable of interest

The observables can be arbitrary Pauli words or linear combinations thereof, `pprop` handles each one independently during propagation.

In [2]:
# Function of parameters
def ansatz(params : list[float]):
    qml.RX(params[0], wires=0)
    qml.RX(params[1], wires=1)

    qml.RY(params[2], wires=0)
    qml.RY(params[3], wires=1)

    qml.Hadamard(wires = 2)

    qml.Barrier()

    qml.CNOT(wires = [0, 1])
    qml.CNOT(wires = [1, 2])

    qml.Barrier()

    qml.RY(params[4], wires=0)
    qml.RY(params[5], wires=1)
    qml.RY(params[6], wires=2)
    
    return (
        [qml.expval(qml.PauliZ(qubit)) for qubit in range(1)]        # ⟨Z₀⟩
        + [qml.expval(qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(2))]  # ⟨X₀X₁X₂⟩
        + [qml.expval(qml.PauliY(2))]                                  # ⟨Y₂⟩
        + [qml.expval(-qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(2) + 13 * qml.PauliZ(2))]  # ⟨-X₀X₁X₂ + 13Z₂⟩
    )

## Step 2: Create the Propagator

`Propagator` takes the ansatz and two optional truncation cutoffs:

- `k1`: **Pauli weight cutoff**: discard any evolved Pauli word with more than `k1` non-identity single-qubit factors. Useful for approximating circuits on many qubits where high-weight terms contribute negligibly.
- `k2`: **Frequency cutoff**: discard any term whose trigonometric degree (total number of sin/cos factors) exceeds `k2`. Controls the complexity of the resulting expression.

Setting both to `None` performs **exact** propagation with no truncation.

In [3]:
prop = Propagator(
    ansatz, 
    k1 = None, # Cutoff on the Pauli Weight
    k2 = None, # Cutoff on the frequencies
)
prop

Propagator
  Number of qubits : 3
  Trainable parameters : 7

## Step 3: Propagate

`.propagate()` evolves each observable backwards through the circuit in the Heisenberg picture. Each line of output shows the initial Pauli word (or linear combination) being propagated.

In [4]:
prop.propagate()

Propagating (1.0000)*Z0
Propagating (1.0000)*X0 X1 X2
Propagating (1.0000)*Y2
Propagating (-1.0000)*X0 X1 X2 + (13.0000)*Z2


## Step 4: Evaluate

Once propagated, `prop` behaves like a plain NumPy function: calling it with a parameter vector returns all expectation values simultaneously as a 1D array of shape `(num_observables,)`.

In [5]:
random_params = qml.numpy.arange(prop.num_params)
prop_output = prop(random_params)
print(prop_output)

[ 0.32448207 -0.5280619   0.          4.16046337]


## Verification against PennyLane

We can verify correctness by running the same circuit as a standard `qml.QNode` and comparing outputs. The two should agree up to floating-point rounding errors (on the order of machine epsilon, ~$10^{-16}$).

In [6]:
device = qml.device('default.qubit', wires = 3)

circuit = qml.QNode(ansatz, device)
pennylane_output = circuit(random_params)
pennylane_output

[tensor(0.32448207, requires_grad=True),
 tensor(-0.5280619, requires_grad=True),
 tensor(1.66533454e-16, requires_grad=True),
 tensor(4.16046337, requires_grad=True)]

The absolute difference between the two outputs confirms that any discrepancy is purely numerical noise, the expressions are analytically identical:

In [7]:
for out1, out2 in zip(prop_output, pennylane_output):
    print(abs(out1 - out2))

5.551115123125783e-17
2.220446049250313e-16
1.6653345369377348e-16
1.7763568394002505e-15


## Inspecting the symbolic expression

`.expression(idx)` reconstructs the closed-form SymPy expression for observable `idx`. This is the actual trigonometric polynomial that `pprop` evaluates numerically — useful for inspecting the structure of the expectation value, checking which parameters contribute, or deriving analytical insights.

Here we inspect the first observable ($\langle Z_0 \rangle$, index 0 by default):

In [8]:
prop.expression()  # defaults to idx=0, i.e. ⟨Z₀⟩

-1.0*sin(θ2)*sin(θ3)*sin(θ4)*cos(θ0)*cos(θ1) + 1.0*cos(θ0)*cos(θ2)*cos(θ4)